In [1]:
%matplotlib inline
%pylab inline

import editdistance

Populating the interactive namespace from numpy and matplotlib


# File Encoding

In [2]:
devanagariChars   = [u'\u0900', u'\u0901', u'\u0902', u'\u0903', u'\u0904', u'\u0905', u'\u0906', u'\u0907', u'\u0908', u'\u0909', u'\u090a', u'\u090b', u'\u090c', u'\u090d', u'\u090e', u'\u090f', u'\u0910', u'\u0911', u'\u0912', u'\u0913', u'\u0914', u'\u0915', u'\u0916', u'\u0917', u'\u0918', u'\u0919', u'\u091a', u'\u091b', u'\u091c', u'\u091d', u'\u091e', u'\u091f', u'\u0920', u'\u0921', u'\u0922', u'\u0923', u'\u0924', u'\u0925', u'\u0926', u'\u0927', u'\u0928', u'\u0929', u'\u092a', u'\u092b', u'\u092c', u'\u092d', u'\u092e', u'\u092f', u'\u0930', u'\u0931', u'\u0932', u'\u0933', u'\u0934', u'\u0935', u'\u0936', u'\u0937', u'\u0938', u'\u0939', u'\u093a', u'\u093b', u'\u093c', u'\u093d', u'\u093e', u'\u093f', u'\u0940', u'\u0941', u'\u0942', u'\u0943', u'\u0944', u'\u0945', u'\u0946', u'\u0947', u'\u0948', u'\u0949', u'\u094a', u'\u094b', u'\u094c', u'\u094d', u'\u094e', u'\u094f', u'\u0950', u'\u0951', u'\u0952', u'\u0953', u'\u0954', u'\u0955', u'\u0956', u'\u0957', u'\u0958', u'\u0959', u'\u095a', u'\u095b', u'\u095c', u'\u095d', u'\u095e', u'\u095f', u'\u0960', u'\u0961', u'\u0962', u'\u0963', u'\u0964', u'\u0965', u'\u0966', u'\u0967', u'\u0968', u'\u0969', u'\u096a', u'\u096b', u'\u096c', u'\u096d', u'\u096e', u'\u096f', u'\u0970', u'\u0971', u'\u0972', u'\u0973', u'\u0974', u'\u0975', u'\u0976', u'\u0977', u'\u0978', u'\u0979', u'\u097a', u'\u097b', u'\u097c', u'\u097d', u'\u097e', u'\u097f']
englishCharacters = [u'a', u'b', u'c', u'd', u'e', u'f', u'g', u'h', u'i', u'j', u'k', u'l', u'm', u'n', u'o', u'p', u'q', u'r', u's', u't', u'u', u'v', u'w', u'x', u'y', u'z']

# LITCM [FIRE 2014] baseline https://github.com/irshadbhat/litcm

In [3]:
from litcm import LIT
lit = LIT(labels=['hin', 'eng'], transliteration=True)

# How to trainsliterate a token

In [4]:
token = 'test'

lit.idfsen = str() 
lit.transliterate(token, 'hin')
print token, lit.idfsen.split('=')[-1][:-1]

test टेस्ट


# CER on different folds

In [6]:
trainCERs = []
testCERs  = []

for fold in range(5):
    
    # File for current fold
    
    testSourceFile = 'folds/testMixed' + str(fold) + '.source.txt'
    trainSourceFile = 'folds/trainMixed' + str(fold) + '.source.txt'
    testTargetFile = 'folds/testMixed' + str(fold) + '.target.txt'
    trainTargetFile = 'folds/trainMixed' + str(fold) + '.target.txt'
    
    # Read training split for current fold
    with open(trainSourceFile, 'r') as fp:
        trainSource = fp.readlines()

    for i in range(len(trainSource)):
        trainSource[i] = trainSource[i].strip('\n')
        string = ''
        for ch in trainSource[i].split(' '):
            if ch != '':
                string += englishCharacters[int(ch)]
        trainSource[i] = string

    with open(testSourceFile, 'r') as fp:
        testSource = fp.readlines()

    for i in range(len(testSource)):
        testSource[i] = testSource[i].strip('\n')
        string = ''
        for ch in testSource[i].split(' '):
            if ch != '':
                string += englishCharacters[int(ch)]
        testSource[i] = string

    # Read testing split for current fold

    with open(trainTargetFile, 'r') as fp:
        trainTarget = fp.readlines()

    for i in range(len(trainTarget)):
        trainTarget[i] = trainTarget[i].strip('\n')
        string = ''
        for ch in trainTarget[i].split(' '):
            if ch != '':
                string += devanagariChars[int(ch)]
        trainTarget[i] = string

    with open(testTargetFile, 'r') as fp:
        testTarget = fp.readlines()

    for i in range(len(testTarget)):
        testTarget[i] = testTarget[i].strip('\n')
        string = ''
        for ch in testTarget[i].split(' '):
            if ch != '':
                string += devanagariChars[int(ch)]
        testTarget[i] = string
    
    # Transliterate using LITCM
    
    trainPredicted = []
    testPredicted  = []

    for token in trainSource:
        lit.idfsen = str() 
        lit.transliterate(token.encode('ascii'), 'hin')
        trainPredicted.append(lit.idfsen.split('=')[-1][:-1].decode('utf-8'))


    for token in testSource:
        lit.idfsen = str() 
        lit.transliterate(token.encode('ascii'), 'hin')
        testPredicted.append(lit.idfsen.split('=')[-1][:-1].decode('utf-8'))
        
    trainCER = 0.
    for i in range(len(trainPredicted)):
        trainCER += (float(editdistance.eval(trainTarget[i], trainPredicted[i])) / len(trainTarget[i]))
    trainCER /= len(trainTarget)
    trainCER *= 100
    
    trainCERs.append(trainCER)
    
    testCER = 0.
    for i in range(len(testPredicted)):
        testCER += (float(editdistance.eval(testTarget[i], testPredicted[i])) / len(testTarget[i]))
    testCER /= len(testTarget)
    testCER *= 100
    
    testCERs.append(testCER)

    print 'Fold :', fold, 'Train data CER:', trainCER, 'Test data CER:', testCER
    
print 'Test  data CER 5-fold :', sum(testCERs) / 5.
print 'Train data CER 5-fold :', sum(trainCERs) / 5.

Fold : 0 Train data CER: 18.8977682789 Test data CER: 18.8640434009
Fold : 1 Train data CER: 18.8083241803 Test data CER: 19.2218629895
Fold : 2 Train data CER: 18.8847477157 Test data CER: 18.9161319418
Fold : 3 Train data CER: 18.8478194843 Test data CER: 19.0638627006
Fold : 4 Train data CER: 19.0192094677 Test data CER: 18.3782199999
Test  data CER 5-fold : 18.8888242065
Train data CER 5-fold : 18.8915738254
